In [111]:
import cv2
import os
import h5py
import json
import numpy as np
from cv2 import dnn
from time import time

# dlib face detection
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import dlib
from imutils import face_utils
from imutils.face_utils import FaceAligner

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, load_model, model_from_json
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.utils import to_categorical
import keras.backend as BK
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, RemoteMonitor
from keras import optimizers

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing

# export the trained model to tensorflowjs compatible version
import tensorflowjs as tfjs

In [112]:
DATA_PATH = "./asl_dataset"
batch_size = 32
dimensions = "rbg"
img_W = 120
img_H = 120
face_cascade = cv2.CascadeClassifier("./dependency/haarcascade_frontalface_default.xml")


In [113]:
DATASET = []

In [114]:
def image_data(imgpath):
    num_people, faces = 0, []
    
    if (dimensions == "rgb"):
        num_people, faces = extract_face(cv2.imread(imgpath))
    if (dimensions == "gray"):
        num_people, faces = extract_face(cv2.imread(imgpath, 0))
        
    if (num_people != 0):
        data = img_to_array(faces[0])
        print(data.shape)
        DATASET.append(data)

In [115]:
def dataset(path):
    if not os.path.exists(path):
        print(f"Error: The path {path} does not exist.")
        return

    dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    num_classes = len(dirs)

    for d in dirs:
        new_path = os.path.join(path, d)
        files = os.listdir(new_path)
        for f in files:
            print(f"Directory: {d}, File: {f}")
            img_path = os.path.join(new_path, f)
            image_data(img_path)

    return num_classes

In [116]:
print(dataset(DATA_PATH))

Directory: k, File: hand2_k_bot_seg_4_cropped.jpeg
Directory: k, File: hand1_k_right_seg_3_cropped.jpeg
Directory: k, File: hand1_k_top_seg_3_cropped.jpeg
Directory: k, File: hand2_k_bot_seg_5_cropped.jpeg
Directory: k, File: hand1_k_bot_seg_3_cropped.jpeg
Directory: k, File: hand3_k_dif_seg_4_cropped.jpeg
Directory: k, File: hand1_k_right_seg_1_cropped.jpeg
Directory: k, File: hand5_k_bot_seg_2_cropped.jpeg
Directory: k, File: hand2_k_top_seg_5_cropped.jpeg
Directory: k, File: hand5_k_dif_seg_5_cropped.jpeg
Directory: k, File: hand5_k_bot_seg_3_cropped.jpeg
Directory: k, File: hand2_k_dif_seg_3_cropped.jpeg
Directory: k, File: hand1_k_bot_seg_5_cropped.jpeg
Directory: k, File: hand2_k_left_seg_4_cropped.jpeg
Directory: k, File: hand2_k_right_seg_3_cropped.jpeg
Directory: k, File: hand2_k_left_seg_3_cropped.jpeg
Directory: k, File: hand2_k_dif_seg_1_cropped.jpeg
Directory: k, File: hand2_k_bot_seg_2_cropped.jpeg
Directory: k, File: hand1_k_left_seg_5_cropped.jpeg
Directory: k, File: ha

In [117]:
def get_training_data():
    num_classes = dataset(DATA_PATH)
    dataset_arr = np.array(DATASET)
    
    num_samples = len(dataset_arr)
    img_p_class = num_samples // num_classes  # Use integer division
    
    start_r = 0
    end_r = img_p_class
    
    label = np.ones((num_samples, ), dtype=int)
    for i in range(0, num_classes):
        label[start_r:end_r] = 1
        start_r = end_r
        end_r += img_p_class
        
    print("Labels shape: ", label)
    return [dataset_arr, label], num_samples, num_classes

In [118]:
def config_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(96, (3, 3), input_shape=input_shape, strides=2))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(96, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(1000, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten()) 
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])


In [119]:
# Save the model
def save_model(model):
    json_string = model.to_jason()
    if not os.path.isdir('model'):
        os.mkdir('model')
        
    open(os.path.join('model', 'model.json'), 'w').write(json_string)
    model.save_weights(os.path.join('model', 'model_weights.h5'), overwrite=True)
    
    tfjs.converters.save_keras_model(model, './model_tfjs')
    print("Model saved.")

In [120]:
def to_categorical(y, num_classes=None):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=np.float32)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [121]:
# Fit the model

def fit_model(input_shape, train_set, labels, num_classes):
    # train the data
    x_train, x_val, y_train, y_val = train_test_split(train_set, labels, test_size=0.2 , random_state=4)
    
    y_train = to_categorical(y_train, num_classes=num_classes)
    y_val = to_categorical(y_val, num_classes=num_classes)
    
    model = config_model(input_shape, num_classes)
    
     # tensorboard
    tensorboard = TensorBoard(
        log_dir='logs/{}'.format(time()), 
        histogram_freq=0, 
        batch_size=batch_size, 
        write_graph=True, 
        write_grads=True, 
        write_images=True, 
        embeddings_freq=0, 
        embeddings_layer_names=None, 
        embeddings_metadata=None
    )

    # early stop if model not improved
    earlystop = EarlyStopping(
        monitor='val_loss', 
        min_delta=0, 
        patience=0, 
        verbose=2, 
        mode='auto'
    )
    
    his = model.fit(
        x_train,
        y_train,
        validation_data=(x_val, y_val),
        batch_size=batch_size,
        epochs=1000,
        shuffle=True,
        callable=[tensorboard]
    )
    
    score = model.evaluating(x_val, y_val, batch_size=2)
    print("score: ", score[0])
    print("accuracy: ", score[1])
    
    save_model(model)

In [122]:
train_data, num_samples, num_classes = get_training_data()

data_t = np.array(train_data[0], dtype="float") / 255.0
label = train_data[1]

BK.set_image_data_format('channels_last')

if (dimensions == "rgb"):
    input_shape = (img_W, img_H, 3)
else:
    input_shape = (img_W, img_H, 1)

print(data_t.shape)
print(input_shape)

fit_model(input_shape, data_t, label, num_classes)

Directory: k, File: hand2_k_bot_seg_4_cropped.jpeg
Directory: k, File: hand1_k_right_seg_3_cropped.jpeg
Directory: k, File: hand1_k_top_seg_3_cropped.jpeg
Directory: k, File: hand2_k_bot_seg_5_cropped.jpeg
Directory: k, File: hand1_k_bot_seg_3_cropped.jpeg
Directory: k, File: hand3_k_dif_seg_4_cropped.jpeg
Directory: k, File: hand1_k_right_seg_1_cropped.jpeg
Directory: k, File: hand5_k_bot_seg_2_cropped.jpeg
Directory: k, File: hand2_k_top_seg_5_cropped.jpeg
Directory: k, File: hand5_k_dif_seg_5_cropped.jpeg
Directory: k, File: hand5_k_bot_seg_3_cropped.jpeg
Directory: k, File: hand2_k_dif_seg_3_cropped.jpeg
Directory: k, File: hand1_k_bot_seg_5_cropped.jpeg
Directory: k, File: hand2_k_left_seg_4_cropped.jpeg
Directory: k, File: hand2_k_right_seg_3_cropped.jpeg
Directory: k, File: hand2_k_left_seg_3_cropped.jpeg
Directory: k, File: hand2_k_dif_seg_1_cropped.jpeg
Directory: k, File: hand2_k_bot_seg_2_cropped.jpeg
Directory: k, File: hand1_k_left_seg_5_cropped.jpeg
Directory: k, File: ha

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.